In [3]:
import sys
sys.path.insert(0, '/projekti/wikidata')
import os
from collections import defaultdict, Counter
from text_utils import remove_reg, tokenise, get_provenance
import json
import numpy as np
from tqdm.notebook import tqdm
from wiki_labels import qid_lab_get

BASE_DIR, DB_DIR = '/projekti/mondoAPI', 'db'

types = json.load(open(os.path.join(BASE_DIR, DB_DIR, 'types.json'), 'r'))
types_a = np.array([tuple([b if b else '' for b in a]) if len(a) == 4 else ('', '', '', '') for a in types],
                   [('tip', 'U3'), ('lang', 'U2'), ('script', 'U4'), ('cc', 'U2')])
t2i = {'_'.join(v):i for i, v in enumerate(types_a)}
print(qid_lab_get(177053, include_alt=True)['mr'])
print(qid_lab_get(191701, include_alt=True)['mba'])

{'en'}
{'nn', 'ru', 'en', 'vi', 'fi', 'tr', 'pt', 'id', 'kk', 'mt', 'cs', 'fa', 'uk', 'lb', 'zh', 'ar', 'hu', 'eu', 'da', 'sv', 'nb', 'my', 'lt', 'fr'}


In [2]:
i = 0
names = defaultdict(Counter)
nema = Counter()
for l in tqdm(open('/backup/wikidata/fnln.txt'), total=26_332_292):
    try:
        qid, n, lang, fn, ln, tip, gender = l.strip('\n').split('\t')
    except Exception as e:
        print(e, l)
    if fn and ln:
        provs = set([a[:-2] for a in get_provenance(n, lang)])
        for prov in provs:
            tag_t = 'fn' + ('1' if gender == 'm' else ('2' if gender == 'f' else '0'))
            tag = tag_t +'_'+prov
            names[fn]['PER_'+tag] += 1
            tag = 'ln' + '_' + prov
            names[ln]['PER_'+tag] += 1
names['prof.'], names['doctor'], names['davor'], names['johnson']

  0%|          | 0/26332292 [00:00<?, ?it/s]

(Counter(),
 Counter({'PER_ln_nn_Latn_': 1,
          'PER_ln_ca_Latn_': 12,
          'PER_ln_sl_Latn_': 4,
          'PER_ln_sq_Latn_': 4,
          'PER_ln_nl_Latn_': 16,
          'PER_ln_es_Latn_': 14,
          'PER_ln_it_Latn_': 1,
          'PER_ln_sv_Latn_': 1,
          'PER_ln_en_Latn_': 16,
          'PER_ln_da_Latn_': 1,
          'PER_ln_nb_Latn_': 1,
          'PER_ln_fr_Latn_': 4,
          'PER_ln_ga_Latn_': 2,
          'PER_ln_de_Latn_': 14,
          'PER_ln_cs_Latn_': 12,
          'PER_ln_pt_Latn_': 2,
          'PER_ln_eo_Latn_': 1,
          'PER_ln_sk_Latn_': 1}),
 Counter({'PER_fn1_pt_Latn_': 12,
          'PER_fn1_ca_Latn_': 7,
          'PER_fn1_sl_Latn_': 9,
          'PER_fn1_fi_Latn_': 6,
          'PER_fn1_nb_Latn_': 6,
          'PER_fn1_sq_Latn_': 6,
          'PER_fn1_nl_Latn_': 13,
          'PER_fn1_es_Latn_': 10,
          'PER_fn1_it_Latn_': 6,
          'PER_fn1_pl_Latn_': 6,
          'PER_fn1_da_Latn_': 6,
          'PER_fn1_en_Latn_': 20,
    

In [5]:
TODO = defaultdict(set)
for l in tqdm(open('data/wikiid.tsv'), total=834281):
    l = l.strip('\n')
    tip, qid = l.strip('\n').split('\t')
    if qid[0] == 'Q':
        if tip in ('TITLE', 'RANK', 'POS',):
            tip = 'tit'
        else:
            tip = tip.lower()

        qid = int(qid[1:])
        TODO[qid].add(tip)
TODO[121594], TODO[177053], len(TODO)

  0%|          | 0/834281 [00:00<?, ?it/s]

({'tit'}, {'tit'}, 790368)

In [14]:
freq_nelma = Counter()
stat = Counter()
for l in tqdm(gzip.open('/backup/wikidata/wikiname.jsonl.gz'), total=10558506):
    j = json.loads(l)
    break
    stat[j['type']] += 1
    if 'title' in j:
        for qtit in j['title']:
            qtit = int(qtit[6:])
            freq_nelma[qtit] += 1
            #if qtit not in TODO:
            #    TODO[qtit].add('tit')
            #elif 'tit' not in TODO[qtit]:
            #    TODO[qtit].add('tit')
    if 'fn' in j:
        for qtit in j['fn']:
            qtit = int(qtit[6:])
            freq_nelma[qtit] += 1
    if 'ln' in j:
        for qtit in j['ln']:
            qtit = int(qtit[6:])
            freq_nelma[qtit] += 1

j

  0%|          | 0/10558506 [00:00<?, ?it/s]

{'type': 'N.TITLE',
 'instance': ['WIKI_Q28640', 'WIKI_Q4164871'],
 'variants': ['WIKI_Q1650915'],
 'oposite_gender': [],
 'nickname': [],
 'soundex': [],
 'cologne': [],
 'native': [],
 'transliteration': [],
 'short_name': [],
 'wiki_id': 'Q901'}

In [7]:
for qid, tips in tqdm(TODO.items(), total=len(TODO)):
    lab = qid_lab_get(qid, include_alt=True)
    for l, langs in lab.items():
        for lang in langs:
            for tip in tips:
                provs = set([a[:-2] for a in get_provenance(l, lang)])
                for prov in provs:
                    tag = f'{tip}_'+prov
                    #print(tag, tag in t2i)
                    if tag in t2i:
                        names[l][f'W_{tag}'] = freq_nelma[qid]
                    else:
                        names[l][f'W_{tip}__unkn_'] = freq_nelma[qid]
                        nema[tag] += 1

names['prof'], names['mba']

  0%|          | 0/798119 [00:00<?, ?it/s]

(Counter({'W_tit_de_Latn_': 4043,
          'W_tit_sl_Latn_': 4043,
          'W_tit_wa_Latn_': 120,
          'W_tit_fr_Latn_': 10}),
 Counter({'W_ln__unkn_': 1,
          'W_fn0__unkn_': 1,
          'W_tit_da_Latn_': 222,
          'W_tit_id_Latn_': 222,
          'W_tit_cs_Latn_': 222,
          'W_tit_en_Latn_': 222,
          'W_tit_sv_Latn_': 222,
          'W_tit_eu_Latn_': 222,
          'W_tit_vi_Latn_': 222,
          'W_tit_hu_Latn_': 222,
          'W_tit_fa_Latn_': 531,
          'W_tit_lb_Latn_': 222,
          'W_tit_ru_Latn_': 222,
          'W_tit_ar_Latn_': 222,
          'W_tit_pt_Latn_': 222,
          'W_tit_nb_Latn_': 222,
          'W_tit_uk_Latn_': 222,
          'W_tit_lt_Latn_': 222,
          'W_tit_my_Latn_': 222,
          'W_tit_tr_Latn_': 222,
          'W_tit_fr_Latn_': 222,
          'W_tit_zh_Latn_': 222,
          'W_tit_kk_Latn_': 222,
          'W_tit_nn_Latn_': 222,
          'W_tit_fi_Latn_': 222,
          'W_tit_he_Latn_': 786}))

In [37]:
toks = defaultdict(Counter)
for tok, v in tqdm(names.items(), total=len(names)):
    for t_prov, cnt in v.items():
        src, t, lang, script, cc = t_prov.split('_')
        tok = tok.replace('_', ' ')
        if len(tok)>0:
            toks[f'{tok}_{lang}_{script}'][(src, t)] += cnt
            if tok[-1] == '.' and tok.count('.') == 1:
                toks[f'{tok[:-1]}_{lang}_{script}'][(src+'.', t)] += cnt
toks = dict(toks)
toks['dr_en_Latn'], toks['mba_en_Latn']

  0%|          | 0/1820542 [00:00<?, ?it/s]

(Counter({('W.', 'tit'): 1486}), Counter({('W', 'tit'): 222}))

In [38]:
t2set = defaultdict(set)
prov2set = defaultdict(set)

for t, c in t2i.items():
    t = t.split('_')
    t2set['_'.join(t[:3])].add(c)    
    prov2set['_'.join(t[1:3])].add('_'.join(t[1:]))
t2set = dict(t2set)    
i2t = {k: v for v, k in t2i.items()}
prov2set = dict(prov2set)    
i2t = {k: v for v, k in t2i.items()}
t2set['tit_hr_Latn'], prov2set['de_Latn']

({1521},
 {'de_Latn_',
  'de_Latn_AT',
  'de_Latn_CH',
  'de_Latn_DE',
  'de_Latn_LI',
  'de_Latn_NL'})

In [39]:
freqs = {}
for k, v in tqdm(toks.items(), total=len(toks)):
    (tok, provt) = k.split('_', 1) 
    for (src, tip), cnt in v.items():
        if (tok, provt) not in freqs:
            freqs[(tok, provt)] = {'fn0': 0, 'fn1': 0, 'fn2': 0, 'ln': 0, 'tit': 0, }
        freqs[(tok, provt)][tip] += cnt

freqs[('prof', 'hr_Latn')]

  0%|          | 0/8314074 [00:00<?, ?it/s]

{'fn0': 0, 'fn1': 0, 'fn2': 0, 'ln': 0, 'tit': 4043}

In [40]:
freqs2 = {}
nema = Counter()
for k, v in tqdm(freqs.items(), total=len(freqs)):
    tok, provt = k
    tips = [k for k in v if v[k]>0]
    if provt in prov2set:
        for tip in tips:
            if tip == 'tit':
                allprovs = [provt+'_']
            else:
                allprovs = [a for a in prov2set[provt] if not a.endswith('_')]
            cnt = v[tip]
            cnt_all = sum(v.values())
            if cnt_all > 0: 
                for prov in allprovs:
                    if f'{tip}_{prov}' in t2i:
                        provi = t2i[f'{tip}_{prov}']
                        freqs2[(tok, provi)] = (cnt, round(cnt*100 / cnt_all,2))
                    else:
                        nema[f'{tip}_{prov}'] += 1
len(nema), sum(nema.values()), freqs2[('doctor', t2i['tit_en_Latn_'])], freqs2[('doctor', t2i['ln_en_Latn_US'])], freqs2[('prof', t2i['tit_hr_Latn_'])]

  0%|          | 0/8314074 [00:00<?, ?it/s]

(20, 1263446, (1, 5.88), (16, 94.12), (4043, 100.0))

In [41]:
# to Oracle
import cx_Oracle
tls = 'db202201301230_high'
connection = cx_Oracle.connect(user='mondo', password=open('orapsw.txt').read(),dsn=tls,encoding="UTF-8")
cur = connection.cursor()


In [6]:
stat

Counter({'loc': 12264207, 'per': 10757820, 'org': 2020191})

In [5]:
import gzip, json
from collections import Counter
from tqdm.auto import tqdm
freq = {}
stat = Counter()
np = {'': 0, '': 1, '':2, '': 3, '':4}
for l in tqdm(gzip.open('/backup/wikidata/wikinelma.jsonl.gz'), total=25_042_218):
    j = json.loads(l)
    stat[j['type']] += 1
    if j['type'] in 
j

  0%|          | 0/20000000 [00:00<?, ?it/s]

{'type': 'per',
 'fn': ['WIKI_Q99578338'],
 'ln': ['WIKI_Q37534081'],
 'pat': [],
 'gender': [],
 'nick': [],
 'title': [],
 'sufix': [],
 'positions': ['WIKI_Q1650915'],
 'educated_at': [],
 'works_at': [],
 'country': [],
 'birth_place': [],
 'death_place': [],
 'residence': [],
 'native_language': [],
 'nationality': [],
 'name_native': [],
 'name_born': [],
 'dob': [],
 'dod': [],
 'picture': [],
 'affiliation': [],
 'wiki_id': 'Q120598284',
 'labels': {'en': ['Samba Dieng'],
  'fr': ['Samba Dieng'],
  'de': ['Samba Dieng']}}

In [42]:
try:
    cur.execute('drop table mondo.wiki_name_parts purge') 
except:
    pass
cur.execute("""create table mondo.wiki_name_parts (token varchar2(200), lang varchar2(10), fn0 number(6, 0), fn1 number(6, 0), fn2 number(6, 0),
            ln number(6, 0), tit number(6, 0), CONSTRAINT pk_locations PRIMARY KEY (token, lang)) ORGANIZATION INDEX""")
print('loading ...')
batch = []
br = 0
for (tok, lng), cnt in tqdm(freqs.items(), total=len(freqs)):
    if len(bytes(tok, 'utf-8')) < 190:
        batch.append((tok, lng, cnt['fn0'], cnt['fn1'], cnt['fn2'], cnt['ln'], cnt['tit'] ))
    if len(batch)>10_048:
        cur.executemany('insert into mondo.wiki_name_parts values (:1, :2, :3, :4, :5, :6, :7) ', batch) 
        connection.commit()
        batch = []
    br += 1
cur.executemany('insert into mondo.wiki_name_parts values (:1, :2, :3, :4, :5, :6, :7) ', batch) 
connection.commit()
print(br)

loading ...


  0%|          | 0/44574635 [00:00<?, ?it/s]

8314074


In [44]:
cur.execute('select count(*) from mondo.wiki_name_parts').fetchall()

[(8313994,)]

In [ ]:
# select m.n, (select t.t from  mondo.TYPEID t where t.id=m.t) t, c from MONDO.NAME_M m where n='dr' and
# not exists (select 1 from mondo.TYPEID where id=m.t and regexp_like(t, 'title_.+') )  and
# exists (select 1 from mondo.wiki_name_parts w where w.token=m.n) and
# not exists (select 1 from mondo.wiki_name_parts w where w.token=m.n and w.PROV=m.T and w.RATIO>5);

# AD HOC cleaning 

cur.execute("""
delete from MONDO.NAME_M m where
not exists (select 1 from mondo.TYPEID where id=m.t and regexp_like(t, 'title_.+') )  and
exists (select 1 from mondo.wiki_name_parts w where w.token=m.n) and
not exists (select 1 from mondo.wiki_name_parts w where w.token=m.n and w.PROV=m.T and w.RATIO>5)
""")
connection.commit()

In [1]:
cur.execute("select m.n, t.t from mondo.name_m m, mondo.TYPEID t where t.id=m.t and m.n='mr.'").fetchall()

NameError: name 'cur' is not defined